In [15]:
import pandas as pd

In [16]:
df = pd.read_csv('data/task-1/train.csv')

In [17]:
# df_dev_1 = pd.read_csv('data/task-1/dev.csv')
# df_dev_2 = pd.read_csv('data/task-2/dev.csv')

In [18]:
df = df.dropna(how="any").reset_index(drop=True)
df = df.drop(['id'], axis=1)
df = df.drop(['grades'], axis=1)

In [19]:
# Define functions for the prepossessing of the data
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def strip_html(data): 
    cleanr = re.compile('<.*?>') 
    cleantext = re.sub(cleanr, ' ', str(data)) 
    return cleantext 

def strip_punc(data): 
    return re.sub('[^A-Za-z]+', ' ', str(data), flags=re.MULTILINE|re.DOTALL) 

def clean(text): 
    
    text = decontracted(text) 
    text = strip_html(text) 
    text = strip_punc(text) 
    
    words=word_tokenize(str(text.lower())) 
    
    # Removing all single letter and stopwords from the question 
    return ' '.join(str(j) for j in words if j not in stop_words and (len(j)!=1))

In [20]:
# Clean the training data
clean_originals = []
clean_edits = []
for _, row in df.iterrows():
    original = str(row['original'])
    start = 0
    end = 0
    for i in range(len(original)):
        if original[i] == '<':
           start = i
        if original[i] == '>' and original[i-1] == '/':
                end = i
    edited = original.replace(original[start:end + 1], row['edit'])
    clean_edits.append(clean(edited))
    original = original[:start] + original[start + 1: end - 1] + original[end + 1:]
    clean_originals.append(clean(original))
df['edited'] = clean_edits
df['original'] = clean_originals
    

In [22]:
# Save the cleaned training data
df.to_csv('data/task-1/train_cleaned.csv', index=False)
